In [ ]:
import string
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
dig = [str(i) for i in range(10)]
cha = list(string.ascii_uppercase[:])
lst = cha + dig

In [ ]:
CATEGORY_MAP = dict()
count = 0
for i in lst:
    CATEGORY_MAP[i]=count
    count = count +1
print(CATEGORY_MAP)

In [ ]:
with open('labels0810.txt','w') as f:
    for i, j in zip(CATEGORY_MAP.keys(), CATEGORY_MAP.values()):
        f.write('{} {}\n'.format(i,j))
    f.close()

In [ ]:
with open('labels0810.txt','r') as f:
    classNames=f.read()
    print(classNames)
    f.close()

In [ ]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

In [ ]:
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import math

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("MODEL_0810.h5", 'labels0810.txt')

offset = 20
imgSize = 300
 
counter = 0
 
labels = lst
 
while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
 
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
 
        imgCropShape = imgCrop.shape
 
        aspectRatio = h / w
 
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.reshape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            imgWhite = cv2.resize(imgWhite, (225, 225))

            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            print(prediction, index)
 
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
 
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
 
 
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
 

    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

In [15]:
imgWhite.shape

(225, 225, 3)